In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
from databases import Database

In [ ]:
df = pd.read_excel('../data/data.xlsx', sheet_name='Sheet1', index_col=0, header=None)
len(df)

In [ ]:
database = Database("sqlite:///../data/data.db")
await database.connect()

In [ ]:
# Create a table.
await database.execute(query="""CREATE TABLE IF NOT EXISTS articles (
                                     id integer PRIMARY KEY,
                                     title text NOT NULL,
                                     publish_date text,
                                     content text
                                 );""")

In [ ]:
# Insert some data.
query = "INSERT INTO articles(title, publish_date, content) VALUES (:title, :publish_date, :content)"
values = []

for i in range(len(df)):
    html_doc = df.iloc[i].name
    soup = BeautifulSoup(html_doc,'html.parser')

    content = []
    for p in soup.body.findAll('p'):
        if p.span:
            content.extend(p.span.contents)
        if p.a:
            content.extend(p.a.contents)
        content.extend(p.contents)
    content = [x for x in content if type(x) == bs4.element.NavigableString]

    # html_doc
    values.append({
        'title': soup.title.contents[0],
        'publish_date': soup.head.find("meta", {"name": "firstpublishedtime"}).get('content'),
        'content': '\n'.join(content),
    })

await database.execute_many(query=query, values=values)

In [ ]:
# Run a database query.
query = "SELECT * FROM articles"

rows = await database.fetch_all(query=query)
len(rows), rows[0]